In [1]:
# kmeans_experiment.ipynb
# K-means from scratch supporting euclidean, 1-cosine, 1-generalized-jaccard
# Assumes data.csv and label.csv are in the working directory (or else you can include the path in the code.)

import time
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

# ---------- helper functions ----------
def row_normalize(A, eps=1e-12):
    norms = np.linalg.norm(A, axis=1, keepdims=True)
    norms = np.maximum(norms, eps)
    return A / norms

def euclidean_distances(X, C):
    X2 = np.sum(X**2, axis=1)[:, None]
    C2 = np.sum(C**2, axis=1)[None, :]
    cross = X @ C.T
    d2 = X2 - 2*cross + C2
    d2 = np.maximum(d2, 0.0)
    return np.sqrt(d2)

def cosine_distance(X, C):
    Xn = row_normalize(X)
    Cn = row_normalize(C)
    sim = Xn @ Cn.T
    sim = np.clip(sim, -1.0, 1.0)
    return 1.0 - sim

def generalized_jaccard_distance(X, C):
    # generalized Jaccard sim = sum min(x_i,y_i) / sum max(x_i,y_i)
    n, d = X.shape
    k = C.shape[0]
    D = np.empty((n, k), dtype=float)
    for j in range(k):
        c = C[j:j+1, :]
        mins = np.minimum(X, c)
        maxs = np.maximum(X, c)
        num = np.sum(mins, axis=1)
        den = np.sum(maxs, axis=1)
        sim = np.where(den == 0, 1.0, num / den)
        D[:, j] = 1.0 - sim
    return D

# ---------- Kmeans implementation ----------
class KMeansCustom:
    def __init__(self, n_clusters=10, metric='euclidean', max_iter=500, tol=1e-6, random_state=0):
        self.n_clusters = n_clusters
        self.metric = metric
        self.max_iter = max_iter
        self.tol = tol
        self.random_state = random_state
        self.centroids = None
        self.inertia_ = None
        self.n_iter_ = 0
        self.history = []
        self.time_taken = None

    def _compute_distances(self, X, C):
        if self.metric == 'euclidean':
            return euclidean_distances(X, C)
        elif self.metric == 'cosine':
            return cosine_distance(X, C)
        elif self.metric == 'jaccard':
            return generalized_jaccard_distance(X, C)
        else:
            raise ValueError("Unknown metric")

    def _init_centroids(self, X):
        rng = np.random.RandomState(self.random_state)
        idx = rng.choice(X.shape[0], self.n_clusters, replace=False)
        return X[idx].astype(float).copy()

    def fit(self, X, stop_on='combined'):
        """
        stop_on:
            'combined' : stop when (centroid change <= tol) OR (SSE increases) OR (max iterations reached)
            'centroid' : stop only when centroids don't change (or max_iter)
            'sse_increase' : stop when SSE increases (or max_iter)
            'maxiter' : stop only at max_iter
        """
        X = X.astype(float)
        C = self._init_centroids(X)
        prev_SSE = None
        self.history = []
        start_time = time.time()

        for it in range(1, self.max_iter + 1):
            D = self._compute_distances(X, C)
            labels = np.argmin(D, axis=1)
            min_dists = D[np.arange(X.shape[0]), labels]
            SSE = np.sum(min_dists**2)
            self.history.append(SSE)

            # centroid update (mean)
            C_new = np.zeros_like(C)
            for j in range(self.n_clusters):
                idxs = np.where(labels == j)[0]
                if len(idxs) == 0:
                    C_new[j] = X[np.random.randint(0, X.shape[0])]
                else:
                    C_new[j] = np.mean(X[idxs], axis=0)

            centroid_shift = np.sqrt(np.sum((C_new - C) ** 2, axis=1))
            max_shift = np.max(centroid_shift)

            stop = False
            if stop_on == 'combined':
                if max_shift <= self.tol:
                    stop = True
                if prev_SSE is not None and SSE > prev_SSE + 1e-12:
                    stop = True
            elif stop_on == 'centroid':
                if max_shift <= self.tol:
                    stop = True
            elif stop_on == 'sse_increase':
                if prev_SSE is not None and SSE > prev_SSE + 1e-12:
                    stop = True
            elif stop_on == 'maxiter':
                stop = False
            else:
                raise ValueError("Unknown stop_on")

            C = C_new
            prev_SSE = SSE

            if stop and stop_on != 'maxiter':
                self.centroids = C
                self.inertia_ = SSE
                self.n_iter_ = it
                break

            if it == self.max_iter:
                self.centroids = C
                self.inertia_ = SSE
                self.n_iter_ = it

        self.time_taken = time.time() - start_time
        return self

    def predict(self, X):
        D = self._compute_distances(X, self.centroids)
        return np.argmin(D, axis=1)


# ---------- cluster accuracy utility ----------
def cluster_accuracy(y_true, cluster_assignments, K):
    mapping = {}
    for j in range(K):
        idx = np.where(cluster_assignments == j)[0]
        if len(idx) == 0:
            mapping[j] = None
        else:
            mapping[j] = Counter(y_true[idx]).most_common(1)[0][0]
    y_pred = np.array([mapping[c] if mapping[c] is not None else -1 for c in cluster_assignments])
    acc = np.mean(y_pred == y_true)
    return acc, mapping


# ---------- script entrypoint ----------
if __name__ == "__main__":
    X = pd.read_csv("data.csv", header=None).values  # 10000 x 784
    y = pd.read_csv("label.csv", header=None).values.ravel()
    K = len(np.unique(y))
    print("Data loaded:", X.shape, "K=", K)

    metrics = ['euclidean', 'cosine', 'jaccard']
    results = {}
    for metric in metrics:
        print(f"\nRunning metric = {metric}")
        km = KMeansCustom(n_clusters=K, metric=metric, max_iter=500, tol=1e-6, random_state=42)
        km.fit(X, stop_on='combined')
        assigns = km.predict(X)
        sse = km.inertia_
        acc, mapping = cluster_accuracy(y, assigns, K)
        results[metric] = {'sse': sse, 'acc': acc, 'iter': km.n_iter_, 'time': km.time_taken}
        print(f"{metric} -> SSE={sse:.6f}, accuracy={acc:.4f}, iterations={km.n_iter_}, time={km.time_taken:.2f}s")

    pd.DataFrame.from_dict(results, orient='index').to_csv("kmeans_results_summary.csv")
    print("\nResults saved to kmeans_results_summary.csv")


Data loaded: (10000, 784) K= 10

Running metric = euclidean
euclidean -> SSE=25414767689.961174, accuracy=0.5851, iterations=33, time=4.44s

Running metric = cosine
cosine -> SSE=686.229294, accuracy=0.6279, iterations=29, time=4.99s

Running metric = jaccard
jaccard -> SSE=4239.946465, accuracy=0.4442, iterations=2, time=2.12s

Results saved to kmeans_results_summary.csv
